# **TumorScope Brain MRI Classification**

In [35]:
import tensorflow as tf
import keras
from PIL import Image

In [36]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [37]:
train_datagen = ImageDataGenerator(rescale =1/255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,)

In [38]:
training_set = train_datagen.flow_from_directory('image_dataset/Training',
                                                 target_size=(64, 64),
                                                 class_mode = 'categorical')

Found 2870 images belonging to 4 classes.


In [39]:
training_set.class_indices

{'glioma_tumor': 0, 'meningioma_tumor': 1, 'no_tumor': 2, 'pituitary_tumor': 3}

In [40]:
test_datagen = ImageDataGenerator(rescale= 1/255)

test_set =test_datagen.flow_from_directory('image_dataset/Testing',
                                           target_size=(64, 64),
                                           class_mode = 'categorical')

Found 377 images belonging to 4 classes.


# Modelling - CNN
Initilising the CNN

In [41]:
from keras.models import Sequential
classifier = Sequential()

**Step 1 - Convolution**

In [42]:
from keras.layers import Conv2D
classifier.add(Conv2D(input_shape=[64,64,3],
                      filters=32, kernel_size=3,
                      activation='relu'))

**Step 2 - Max Pooling**

In [43]:
from keras.layers import MaxPooling2D
classifier.add(MaxPooling2D(pool_size=2,strides=2))

**Step 3 -Flattening**

In [44]:
from keras.layers import Flatten
classifier.add(Flatten())

**Step 4 - Full Connection**

In [45]:
from keras.layers import Dense

# Hidden layers with 128 units and ReLU activation
classifier.add(Dense(units = 64, activation='relu'))

# Output layer with 1 unit and softmax activation
classifier.add(Dense(units = 4, activation='softmax'))

**Training the CNN Model**


In [46]:
classifier.compile(optimizer= 'adam',
                   loss = 'categorical_crossentropy',
                   metrics = ['accuracy'])

In [47]:
classifier.fit(
    x=training_set,
    validation_data=test_set,
    epochs=55
)


Epoch 1/55
90/90 [==============================] - 6s 67ms/step - loss: 1.1779 - accuracy: 0.4826 - val_loss: 1.4870 - val_accuracy: 0.4005
Epoch 2/55
90/90 [==============================] - 6s 63ms/step - loss: 0.8764 - accuracy: 0.6491 - val_loss: 1.6692 - val_accuracy: 0.4562
Epoch 3/55
90/90 [==============================] - 6s 62ms/step - loss: 0.7330 - accuracy: 0.7045 - val_loss: 1.8779 - val_accuracy: 0.5385
Epoch 4/55
90/90 [==============================] - 6s 64ms/step - loss: 0.6441 - accuracy: 0.7463 - val_loss: 1.9835 - val_accuracy: 0.5464
Epoch 5/55
90/90 [==============================] - 6s 69ms/step - loss: 0.5873 - accuracy: 0.7714 - val_loss: 1.7743 - val_accuracy: 0.5570
Epoch 6/55
90/90 [==============================] - 6s 62ms/step - loss: 0.5410 - accuracy: 0.7861 - val_loss: 2.1762 - val_accuracy: 0.5358
Epoch 7/55
90/90 [==============================] - 5s 61ms/step - loss: 0.5310 - accuracy: 0.7955 - val_loss: 1.8665 - val_accuracy: 0.6127
Epoch 8/55
90

# Evalution
- Making a single prediction

In [51]:
import numpy as np
from PIL import Image

In [ ]:
import streamlit as st
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model

# Load the model (adjust the path to your model)
classifier = load_model('path_to_your_model.h5')  # Replace with your model's path

# Streamlit Web App Interface
st.title("Brain Tumor MRI Classification")
st.write("Upload an MRI image to classify the tumor type.")

# File uploader
uploaded_image = st.file_uploader("Choose an MRI image...", type=["jpg", "jpeg", "png"])

if uploaded_image is not None:
    # Open the uploaded image
    test_image = Image.open(uploaded_image)
    
    # Data Preprocessing
    test_image = test_image.resize((64, 64))  # Resize to match the model's input size
    test_image = np.array(test_image)  # Convert image to numpy array
    test_image = test_image / 255.0  # Normalize the image (if this was done during training)
    test_image = np.expand_dims(test_image, axis=0)  # Add batch dimension

    # Prediction
    result = classifier.predict(test_image)

    # Get the predicted class
    predicted_class = np.argmax(result, axis=1)

    # Display the result
    if predicted_class == 0:
        st.image(uploaded_image, caption="Glioma Tumor", use_column_width=True)
        st.write("The image is classified as a Glioma Tumor.")
    elif predicted_class == 1:
        st.image(uploaded_image, caption="Meningioma Tumor", use_column_width=True)
        st.write("The image is classified as a Meningioma Tumor.")
    elif predicted_class == 2:
        st.image(uploaded_image, caption="No Tumor", use_column_width=True)
        st.write("The image is classified as No Tumor.")
    else:
        st.image(uploaded_image, caption="Pituitary Tumor", use_column_width=True)
        st.write("The image is classified as a Pituitary Tumor.")


1/1 [==============================] - 0s 13ms/step
The image is a Meningioma Tumor
1/1 [==============================] - 0s 13ms/step
The image is a Meningioma Tumor


In [53]:
# Save the model to a file
classifier.save('brain_tumor_model.h5')